# Resnet18 Model Using Internal Data

In [9]:
%reload_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append("..")

from src.data.prepare_data import *
from src.models.model import *
import time
import random
from sklearn.model_selection import KFold
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data.sampler import WeightedRandomSampler
from tqdm import tqdm

In [11]:
SEED = 2718
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [12]:
train = pd.read_csv("../data/internal/train.csv")
test = pd.read_csv("../data/internal/test.csv")
sub = pd.read_csv("../data/internal/sample_submission.csv")

This model is large enough that we will need to use smaller images on my local machine.

In [1]:
config = {
    'INPUT_DIR'      : '',
    'MODEL'          : 'resnet18',
    'SIZE'           : 64,
    'BATCH_SIZE'     : 128,
    'NUM_FOLDS'      : 3,
    'NUM_EPOCHS'     : 10,
    'FREEZED_EPOCHS' : 3,
    'LEARNING_RATE'  : 1e-3,
    'EARLY_STOPPING' : 3,
    'UNIFORM_AUGMENT': True,
    'TTA'            : 3,
    'NUM_WORKERS'    : 16,
    'DEVICE'         : 'cpu'
}

In [14]:
t = time.time()
predictions = pd.DataFrame()
transform = ImageTransform(config['SIZE'], config['UNIFORM_AUGMENT'])

skf = KFold(n_splits=config['NUM_FOLDS'], shuffle=True, random_state=SEED)
for i, (idxT,idxV) in enumerate(skf.split(np.arange(15))):
    t_fold = time.time()
    tr = train.loc[train.tfrecord.isin(idxT)]
    va = train.loc[train.tfrecord.isin(idxV)]
    tr.reset_index(drop=True, inplace=True)
    va.reset_index(drop=True, inplace=True)

    # create datasets
    dataset_train = MelanomaDataset("../data/internal/train", tr, transform=transform, phase='train')
    dataset_valid = MelanomaDataset("../data/internal/train", va, transform=transform, phase='valid')
    
    # load a pretrained model
    net = load_model(config['MODEL'], 2)

    # define a loss function
    criterion = nn.CrossEntropyLoss()

    # define an optimizer
    optimizer = optim.Adam(net.parameters(), lr=config['LEARNING_RATE'])

    # define a scheduler
    scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=2, factor=0.2)

    # create a sampler
    sampler = create_weighted_random_sampler(tr)

    # train the network
    print(f"---- fold: {i + 1} ------------")
    train_model(
        f"{config['MODEL']}_{i + 1}",
        dataset_train,
        dataset_valid,
        config['BATCH_SIZE'],
        net,
        criterion,
        optimizer,
        scheduler,
        config['NUM_EPOCHS'],
        config['FREEZED_EPOCHS'],
        config['INPUT_DIR'],
        config['NUM_WORKERS'],
        sampler,
        config['DEVICE'],
        config['EARLY_STOPPING']
    )

    # predict on test dataset
    test['target'] = 0
    dataset_test = MelanomaDataset("../data/internal/test", test, transform=transform, phase='test')
    predictions = get_predictions(dataset_test, 
                                  config["BATCH_SIZE"], 
                                  net, 
                                  config["TTA"], 
                                  predictions, 
                                  config["DEVICE"])
    predictions.to_csv(f'../submissions/{config["MODEL"]}_fold{i+1}.csv')
    print(f"fold took {round(time.time() - t_fold, 2)}")
    
# output
sub['target'] = predictions.mean(axis=1)
sub.to_csv(f"../submissions/submission{config['MODEL']}.csv", index=False)
print(f"total time: {round(time.time() - t, 4)}")

  0%|          | 0/171 [00:00<?, ?it/s]

---- fold: 1 ------------


  0%|          | 0/171 [00:00<?, ?it/s]

epoch: 1, loss_train: 0.6177, loss_valid: 0.7451, auc_valid: 0.6298, saved: True, 263.2864sec


  0%|          | 0/171 [00:00<?, ?it/s]

...

epoch: 9, loss_train: 0.1715, loss_valid: 0.0878, auc_valid: 0.8281, saved: False, 575.6141sec


  0%|          | 0/3 [00:00<?, ?it/s]

epoch: 10, loss_train: 0.1596, loss_valid: 0.0931, auc_valid: 0.7748, saved: False, 576.2096sec


100%|██████████| 3/3 [03:07<00:00, 62.38s/it]

fold took 4796.69
total time: 12602.5304


This submission using only 64x64 images and a ResNet18 architecture scored 0.8780